# Inside the Iceberg: Metadata structures

In the previous notebook, we saw Iceberg from the outside - creating tables, appending data, querying. Now let's look inside to understand exactly how Iceberg works.

We'll explore:

```
The catalog database: What's actually inside the SQLite file?
  └─ Metadata file (JSON): Schema, snapshots, and table history
       └─ Manifest List (AVRO): What's inside a snapshot?
            └─ Manifest (AVRO): Lists of data files and statistics
                └─ Data files (Parquet): The actual data
```

and finally look at the complete picture once more. By the end, you'll understand how Iceberg achieves atomic commits, time travel, and fast queries.

In [ ]:
import shutil
import daft
import pyarrow as pa
from pathlib import Path
from pyiceberg.catalog.sql import SqlCatalog
import sqlite3
from datetime import datetime

%reload_ext autoreload
%autoreload 2
from helpers import inspect_metadata_json, inspect_manifest, inspect_manifest_list

## Setup: Create a table with history

First, let's create a table with multiple snapshots so we have interesting metadata to explore.

In [ ]:
warehouse_path = Path('../data/warehouse_metadata').absolute()
shutil.rmtree(warehouse_path, ignore_errors=True)
warehouse_path.mkdir(parents=True, exist_ok=True)
catalog_db = warehouse_path / 'catalog.db'
catalog_db.unlink(missing_ok=True)

catalog = SqlCatalog(
    'metadata_demo',
    **{'uri': f'sqlite:///{catalog_db}', 'warehouse': f'file://{warehouse_path}'}
)
catalog.create_namespace('demo')
print("✅ Catalog initialized")

In [ ]:
# Load events data and create table with multiple operations
df_events = daft.read_json('../data/input/events.jsonl')

# Snapshot 1: Initial data
print("Snapshot 1: Creating initial load...")
df_batch1 = df_events.limit(30000)
arrow_table = df_batch1.to_arrow()
events_table = catalog.create_table('demo.events', schema=pa.schema(arrow_table.schema))
events_table.append(arrow_table)
print(f"Snapshot 1: Appended {len(arrow_table):,} records")

# Snapshot 2: Append more
print("Snapshot 2: Appending more data...")
df_batch2 = df_events.offset(30000).limit(30000)
arrow_table = df_batch2.to_arrow()
events_table.append(arrow_table)
print(f"Snapshot 2: Appended {len(arrow_table):,} more records")

# Snapshot 3: Delete some records
print("Snapshot 3: Deleting OperationMode events...")
events_table.delete("type = 'OperationMode'")
print("Snapshot 3: Deleted OperationMode events")

print(f"\n✅ Created table with {len(events_table.history())} snapshots")

## The catalog database

```
The catalog database: What's actually inside the SQLite file? ← We are here!
  └─ Metadata file (JSON): Schema, snapshots, and table history
       └─ Manifest List (AVRO): What's inside a snapshot?
            └─ Manifest (AVRO): Lists of data files and statistics
                └─ Data files (Parquet): The actual data
```

The catalog database is the **entry point** to all Iceberg tables. It's in our case a simple SQLite database that stores:

* **Table locations**: Where each table's metadata lives
* **Namespace properties**: Configuration for database schemas
* **Atomic pointers**: Current metadata file for each table

### Why use a catalog?

The catalog enables **atomic commits**. When a writer updates a table, the catalog

1. Writes a new metadata JSON file
2. Updates the catalog pointer atomically (SQL UPDATE)
3. In case of conflicts, returns an error and the client has to retry (optimistic concurrency)

The catalog is the **single source of truth** for which metadata file is current.

### Inspecting the catalog

Let's look inside the SQLite database:

In [ ]:
conn = sqlite3.connect(catalog_db)
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()
print("Tables in catalog database:")
for table in tables:
    print(f"  • {table[0]}")

In [ ]:
# Show the schema of iceberg_tables
cursor.execute("PRAGMA table_info(iceberg_tables)")
columns = cursor.fetchall()
print("Schema of 'iceberg_tables':")
for col in columns:
    print(f"  {col[1]}: {col[2]}")

cursor.execute("PRAGMA table_info(iceberg_namespace_properties)")
columns = cursor.fetchall()
print("\nSchema of 'iceberg_namespace_properties':")
for col in columns:
    print(f"  {col[1]}: {col[2]}")


In [ ]:
cursor.execute("SELECT * FROM iceberg_tables")
rows = cursor.fetchall()

print("Registered Iceberg tables")
for row in rows:
    catalog_name, namespace, table_name, metadata_location, prev_metadata = row
    print(f"{namespace}.{table_name}")
    print(f"  Current metadata: {Path(metadata_location).name}")
    if prev_metadata:
        print(f"  Previous metadata: {Path(prev_metadata).name}")
    print()

cursor.execute("SELECT * FROM iceberg_namespace_properties")
rows = cursor.fetchall()

print("Registered namespaces:")
for row in rows:
    catalog_name, namespace, key, value = row
    print(f"{catalog_name}.{namespace}")
    print(f"  {key}: {value}")

### What we see

The `iceberg_tables` table has:

* **metadata_location**: Points to the **current** metadata JSON file
* **previous_metadata_location**: Points to the **previous** metadata JSON file

This is how Iceberg achieves **atomic commits**:

```sql
UPDATE iceberg_tables
SET metadata_location = 'new_metadata.json',
    previous_metadata_location = 'old_metadata.json'
WHERE table_name = 'events'
  AND metadata_location = 'old_metadata.json'  -- Optimistic lock!
```

If two writers try to commit at the same time:
- First succeeds (updates the row)
- Second fails (WHERE clause doesn't match anymore)
- Second must retry with the new metadata

This is **optimistic concurrency control**!

In [ ]:
conn.close()

## Metadata file (JSON)

```
The catalog database: What's actually inside the SQLite file?
  └─ Metadata file (JSON): Schema, snapshots, and table history ← We are here!
       └─ Manifest List (AVRO): What's inside a snapshot?
            └─ Manifest (AVRO): Lists of data files and statistics
                └─ Data files (Parquet): The actual data
```

The metadata file represents a version of an Iceberg table. Each commit creates a **new JSON file**. This file contains:

* **Schema versions**: All schema versions (for time travel)
* **Partition specs**: All partition specs (for partition evolution)
* **Snapshots**: All snapshots with their manifest lists
* **Snapshot log**: Chronological list of snapshots
* **Current snapshot ID**: Pointer to the current snapshot
* **Metadata log**: History of metadata files

Let's find and inspect a metadata JSON file. 

In [ ]:
# Find metadata files
table_dir = Path(events_table.location().replace('file://', ''))
metadata_files = sorted(table_dir.glob('metadata/*.metadata.json'))

print(f"Found {len(metadata_files)} metadata file(s):")
for i, mf in enumerate(metadata_files, 1):
    size = mf.stat().st_size
    print(f"  {i}. {mf.name} ({size:,} bytes, {size/1024:.1f} KB)")

# Use the latest metadata file
latest_metadata = metadata_files[-1]
print(f"\nUsing latest: {latest_metadata.name}")

There is a lot of stuff in the file. Here is a visualization of the contents:

In [ ]:
metadata = inspect_metadata_json(latest_metadata)

### Schemas

Iceberg stores **all schema versions** in the metadata. Each schema has a unique ID.

When you read a snapshot, Iceberg uses the schema that was current at that snapshot. This enables:
* **Time travel with old schemas**
* **Schema evolution without rewrites**

### Snapshots

Each snapshot represents a **commit** to the table. Snapshots contain:

* **snapshot-id**: Unique identifier
* **timestamp-ms**: When this snapshot was created
* **manifest-list**: Path to AVRO file listing manifests
* **schema-id**: Which schema version to use
* **summary**: Statistics (operation, files added/deleted, records added/deleted)

### Logs

In addition (not shown in the visualization):

* The `snapshot-log` is a chronological list of snapshots with timestamps. This enables:
  * **Time travel by timestamp**: "Show me data as of 2024-12-01"
  * **Audit trail**: When was each commit made?
* The `metadata-log` tracks which metadata files existed and when. This is used for:
  * **Metadata file expiration**: Clean up old metadata files
  * **Debugging**: Understand table history
  * **Consistency checks**: Verify metadata chain

The logs are kept separate because entries in `snapshots` may be cleaned up separately as we will see.

Can you guess from the metadata how the delete operation in the third snapshot was handled by Daft? 

Feel free to open the JSON file directly.

## Manifest lists (AVRO)

```
The catalog database: What's actually inside the SQLite file?
  └─ Metadata file (JSON): Schema, snapshots, and table history
       └─ Manifest List (AVRO): What's inside a snapshot? ← We are here!
            └─ Manifest (AVRO): Lists of data files and statistics
                └─ Data files (Parquet): The actual data
```

Let's continue our trace. Each snapshot in the metadata JSON points to a **manifest list** file. The manifest list is itself a binary [AVRO](https://avro.apache.org/) file that contains references to individual manifest files.

The manifest list acts as an **index of indices** - it tracks which manifest files are part of this snapshot, along with statistics about each manifest (like number of files, number of rows, partition info).

It provides several important capabilities:

* **Snapshot isolation**: Multiple snapshots can share the same manifest files. The manifest list specifies which manifests belong to this snapshot.
* **Aggregated statistics**: Total file counts and row counts per manifest enable quick query planning without reading all manifests.
* **Partition pruning**: For partitioned tables, manifest lists contain partition bounds to skip entire manifests based on filters.
* **Content tracking**: Separate manifests for data files (content=0) and delete files (content=1) are tracked together.

Let's check out what is inside.

In [ ]:
# Extract the current snapshot from the metadata
current_snapshot_id = metadata['current-snapshot-id']
current_snapshot = next(s for s in metadata['snapshots'] if s['snapshot-id'] == current_snapshot_id)

# Extract manifest list path
manifest_list_path = current_snapshot['manifest-list']
manifest_list_file = Path(manifest_list_path.replace('file://', ''))

# Visualize and capture the entries for later use
manifest_list_entries = inspect_manifest_list(manifest_list_file, latest_metadata.name, current_snapshot_id)

## Manifest files (AVRO)

```
The catalog database: What's actually inside the SQLite file?
  └─ Metadata file (JSON): Schema, snapshots, and table history
       └─ Manifest List (AVRO): What's inside a snapshot?
            └─ Manifest (AVRO): Lists of data files and statistics ← We are here!
                └─ Data files (Parquet): The actual data
```

Now let's look at the individual manifest files referenced by the manifest list. Each manifest file contains the actual list of data files (Parquet files) with detailed statistics.
* **Data file paths**: Where the Parquet files are
* **Partition values**: What partition each file belongs to
* **Statistics**: Record counts, min/max values, null counts
* **File metadata**: Size, format, compression
* **Status**: Whether the file is ADDED, EXISTING, or DELETED

This metadata enables **predicate pushdown** - skipping files without reading them.


In [ ]:
# Extract manifest paths from the manifest list and collect data file paths
print(f"The manifest list references {len(manifest_list_entries)} manifest file(s).")

# Collect all data file paths from all manifests
all_data_file_paths = []
for i, entry in enumerate(manifest_list_entries, 1):
    manifest_path = Path(entry['manifest_path'].replace('file://', ''))
    data_file_paths = inspect_manifest(manifest_path)
    all_data_file_paths.extend(data_file_paths)


## Data files (Parquet)

```
The catalog database: What's actually inside the SQLite file?
  └─ Metadata file (JSON): Schema, snapshots, and table history
       └─ Manifest List (AVRO): What's inside a snapshot?
            └─ Manifest (AVRO): Lists of data files and statistics
                └─ Data files (Parquet): The actual data ← We are here!
```

Finally, the actual data! Data files are standard **Parquet files**. Iceberg doesn't change Parquet - it just tracks them in manifests. It makes sure that you always see valid, complete files and no stale data.


## The complete picture: Tracing a query

Now let's trace how a query uses all these metadata structures:

```
SELECT * FROM events WHERE type = 'OperationMode' AND time > '2026-01-01'
```

### Step-by-step query execution

1. **Catalog lookup** (SQLite)
   - Query: `SELECT metadata_location FROM iceberg_tables WHERE table_name = 'events'`
   - Result: Path to current metadata JSON file

2. **Read metadata JSON**
   - Parse: `current-snapshot-id`
   - Find snapshot with that ID
   - Get: `manifest-list` path

3. **Read manifest list** (AVRO)
   - Lists all manifest files for this snapshot
   - Each manifest has aggregated statistics
   - Can skip entire manifests based on partition bounds

4. **Read manifests** (AVRO)
   - For each manifest (that wasn't skipped), check file-level statistics:
     - Does `lower_bounds['type']` ≤ 'OperationMode' ≤ `upper_bounds['type']`?
     - Does `lower_bounds['time']` ≤ '2026-01-01' ≤ `upper_bounds['time']`?
   - If not: **skip this data file entirely**
   - If yes: add file to the read list

5. **Read data files** (Parquet)
   - Read only files that passed predicate pushdown
   - Within each file, read only necessary columns
   - Apply row-level filters

This is why Iceberg is fast. 

* Everything outside of the catalog database is only written once and can be safely cached. 
* It reads minimal metadata to skip most of the data!

### Visualizing the hierarchy

In [ ]:
print("Complete Iceberg metadata hierarchy:\n")
print("1. 📚 Catalog (SQLite)")
print(f"   {catalog_db.name}")
print(f"   └─ Table: demo.events → {latest_metadata.name}")
print()
print("2. 📄 Metadata JSON")
print(f"   {latest_metadata.name}")
print(f"   ├─ Schema: {len(metadata['schemas'])} version(s)")
print(f"   ├─ Partition specs: {len(metadata['partition-specs'])}")
print(f"   └─ Snapshots: {len(metadata['snapshots'])}")
print()
print("3. 📋 Manifest List (AVRO)")
print(f"   {manifest_list_file.name} ({manifest_list_file.stat().st_size:,} bytes)")
print(f"   └─ References {len(manifest_list_entries)} manifest file(s)")
print()
print("4. 📦 Manifest Files (AVRO)")
for i, entry in enumerate(manifest_list_entries, 1):
    manifest_path = Path(entry['manifest_path'].replace('file://', ''))
    print(f"   {manifest_path.name} ({manifest_path.stat().st_size:,} bytes)")
print()
print("5. 💾 Data Files (Parquet)")
for i, df in enumerate(all_data_file_paths, 1):
    print(f"   {df.name} ({df.stat().st_size / 1024 / 1024:.2f} MB)")
print()

# Calculate overhead including manifest list
manifest_list_size = manifest_list_file.stat().st_size
manifest_files_size = sum(Path(entry['manifest_path'].replace('file://', '')).stat().st_size
                          for entry in manifest_list_entries)
total_metadata_size = sum(mf.stat().st_size for mf in metadata_files) + manifest_list_size + manifest_files_size
print(f"Total metadata overhead: {total_metadata_size / 1024:.1f} KB")
print(f"  - Metadata JSON: {sum(mf.stat().st_size for mf in metadata_files) / 1024:.1f} KB")
print(f"  - Manifest list: {manifest_list_size / 1024:.1f} KB")
print(f"  - Manifest files: {manifest_files_size / 1024:.1f} KB")
print(f"Total data size: {sum(df.stat().st_size for df in all_data_file_paths) / 1024 / 1024:.2f} MB")

## Review questions

**Why store metadata in multiple JSON files instead of one?**
 - Think about atomicity and append-only operations.

**What would happen if you directly edited a data file?**
 - Would the manifest notice? Would queries see your changes?

**How does Iceberg achieve atomic commits?**
 - What SQL statement is used? What makes it atomic?

**Why have both manifest lists and manifest files?**
 - What additional capabilities does the manifest list provide?

**How does predicate pushdown work across the metadata layers?**
 - At what levels can files/partitions be skipped?

**What's the relationship between the schema in Parquet files and the schema in Iceberg metadata?**
 - Can they differ? What might happen if they do?


## Challenges

### Be the janitor and clean up 

1. Open the latest metadata JSON in a text editor and find the `current-snapshot-id`
3. Locate that snapshot in the `snapshots` array and extract the `manifest-list` path
5. Verify that this file exists in the metadata directory and list it using `avro-tools` (see last section).
6. Locate the manifest paths in the manifest list and open each using `avro-tools`.
7. Locate the referenced data files.
8. Determine what can files and metadata can be deleted if you just want to preserve the latest snapshot.

### Analyze metadata overhead

1. Load 100 rows of radiator data into an Iceberg table.
2. Calculate the total amount of metadata stored in JSON, AVRO and Parquet files. 
3. Calculate the net data amount stored in the Parquet files (dictionary and data pages).
4. Calcuate the ratio of metadata and data size. 
5. Repeat for 1000, 10000, 100000 rows. What are economic sizes for batches of data?



## Summary

In this deep dive, we explored the complete Iceberg metadata hierarchy:

* **Catalog database**: The atomic pointer to current metadata
  - Enables optimistic concurrency control
  - Single UPDATE statement makes commits atomic

* **Metadata JSON**: Complete table state
  - All schema versions (for time travel)
  - All snapshots with manifest lists
  - Snapshot log for temporal queries
  - Metadata log for file management

* **Manifest lists**: Index of manifest files
  - AVRO format for efficiency
  - Aggregated statistics per manifest
  - Enables manifest-level pruning
  - Tracks file status across snapshots

* **Manifest files**: Index of data files
  - AVRO format with detailed statistics
  - Per-file record counts and bounds
  - Partition information
  - Enables fine-grained predicate pushdown

* **Data files**: Immutable Parquet
  - Never modified after creation
  - Referenced by manifests
  - Standard Parquet format

### Key insights

1. **Layered metadata**: Each layer provides increasingly detailed information
2. **Metadata is append-only**: New files created, old ones retained
3. **Multiple pruning levels**: Skip entire manifests or individual files
4. **Statistics enable pruning**: At manifest list, manifest, and file level
5. **Everything is versioned**: Time travel works by reading old snapshots
6. **Minimal metadata overhead**: ~KB of metadata per GB of data

In the next section, we'll look at how the metadata structure supports time travel. 
